In [1]:
import pandas as pd
import numpy as np
import math
import statistics
import sklearn
import math

In [2]:
import seaborn as sns

In [3]:
from sklearn.model_selection import train_test_split
from category_encoders import BinaryEncoder, TargetEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

**Splitting training and testing data**

In [4]:
data = pd.read_csv('data.csv')
X = data.drop(['satisfaction'], axis=1)
Y = data[['satisfaction']]

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train)
X_Y_train = X_train.join(Y_train)

**Data analysis**

In [6]:
numericFactorsWithZero = ['Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking', 'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort', 'Inflight entertainment', 'On-board service', 'Leg room service', 'Baggage handling', 'Checkin service', 'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes']

In [7]:
X_Y_train.corr()

C:\Users\logis\AppData\Local\Temp\ipykernel_21872\344315668.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  X_Y_train.corr()


,Age,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
Age,1.000000,0.102931,0.014524,0.029797,0.020989,0.002146,0.023112,0.207109,0.158909,0.075175,0.055534,0.041092,-0.047678,0.030700,0.050731,-0.008295,-0.011194,0.134927
Flight Distance,0.102931,1.000000,0.006677,-0.022683,0.063146,0.004069,0.056765,0.219058,0.157682,0.131919,0.114072,0.135679,0.066751,0.077710,0.098798,-0.000746,-0.004661,0.299153
Inflight wifi service,0.014524,0.006677,1.000000,0.344631,0.712465,0.338559,0.129325,0.455701,0.120999,0.208695,0.119790,0.162518,0.123253,0.043170,0.131698,-0.014483,-0.017830,0.279439
Departure/Arrival time convenient,0.029797,-0.022683,0.344631,1.000000,0.440564,0.445748,-0.001220,0.068483,0.007472,-0.011271,0.063737,0.006269,0.069799,0.088732,0.009470,0.004266,0.001001,-0.059550
Ease of Online booking,0.020989,0.063146,0.712465,0.440564,1.000000,0.465468,0.026598,0.400923,0.029310,0.043735,0.036610,0.107791,0.041025,0.008912,0.015677,-0.005017,-0.008421,0.161271
Gate location,0.002146,0.004069,0.338559,0.445748,0.465468,1.000000,-0.003846,0.004270,0.004277,-0.000531,-0.033290,-0.009521,0.001983,-0.041445,-0.005278,0.003156,0.001682,-0.006424
Food and drink,0.023112,0.056765,0.129325,-0.001220,0.026598,-0.003846,1.000000,0.234345,0.572563,0.620499,0.055533,0.030416,0.034320,0.080370,0.655223,-0.030185,-0.031608,0.209936
Online boarding,0.207109,0.219058,0.455701,0.068483,0.400923,0.004270,0.234345,1.000000,0.419923,0.285126,0.156669,0.127846,0.084321,0.206249,0.327517,-0.021667,-0.025120,0.503832
Seat comfort,0.158909,0.157682,0.120999,0.007472,0.029310,0.004277,0.572563,0.419923,1.000000,0.609412,0.131483,0.101929,0.075249,0.186429,0.676408,-0.029718,-0.030624,0.348724
Inflight entertainment,0.075175,0.131919,0.208695,-0.011271,0.043735,-0.000531,0.620499,0.285126,0.609412,1.000000,0.420798,0.301117,0.383557,0.117465,0.692236,-0.024110,-0.026355,0.399924


In [8]:
X_Y_train.groupby('Class', as_index=False) \
    .aggregate({'satisfaction': 'mean'})

,Class,satisfaction
0,Business,0.696543
1,Eco,0.188876
2,Eco Plus,0.245889


In [9]:
X_Y_train.groupby('Gender', as_index=False) \
    .aggregate({'satisfaction': 'mean'})

,Gender,satisfaction
0,Female,0.429499
1,Male,0.441998


In [10]:
numeric_factors_to_use = ['Flight Distance', 'Inflight wifi service', 'Online boarding', 'Seat comfort', 'Inflight entertainment', 'On-board service', 'Leg room service', 'Checkin service']

In [11]:
categorial_factors_to_use = ['Customer Type', 'Type of Travel', 'Class']
factors_to_use = numeric_factors_to_use + categorial_factors_to_use
print(factors_to_use)

['Flight Distance', 'Inflight wifi service', 'Online boarding', 'Seat comfort', 'Inflight entertainment', 'On-board service', 'Leg room service', 'Checkin service', 'Customer Type', 'Type of Travel', 'Class']


In [12]:
X_train = X_train[factors_to_use]
X_valid = X_valid[factors_to_use]
X_test = X_test[factors_to_use]

In [13]:
factors_with_zero_to_use = ['Inflight wifi service', 'Online boarding', 'Seat comfort', 'Inflight entertainment', 'On-board service', 'Leg room service', 'Checkin service']

**Encoding and scaling**

In [14]:
for categor_factor in factors_with_zero_to_use:
    encode = TargetEncoder()
    encode.fit(X_train[categor_factor], Y_train)
    s = categor_factor + "_encoded"
    X_train[s] = encode.transform(X_train[categor_factor])

X_train = X_train.drop(factors_with_zero_to_use, axis='columns')


['Customer Type', 'Type of Travel', 'Class']

In [15]:
for categor_factor in factors_with_zero_to_use:
    s = categor_factor + "_encoded"
    X_valid[s] = encode.transform(X_valid[categor_factor])
    X_test[s] = encode.transform(X_test[categor_factor])

X_valid = X_valid.drop(factors_with_zero_to_use, axis='columns')
X_test = X_test.drop(factors_with_zero_to_use, axis='columns')

In [16]:
Customer_encoder = TargetEncoder()
Customer_encoder.fit(X_train['Customer Type'], Y_train)
X_train['Customer Type'] = Customer_encoder.transform(X_train['Customer Type'])
X_valid['Customer Type'] = Customer_encoder.transform(X_valid['Customer Type'])
X_test['Customer Type'] = Customer_encoder.transform(X_test['Customer Type'])

Travel_encoder = TargetEncoder()
Travel_encoder.fit(X_train['Type of Travel'], Y_train)
X_train['Type of Travel'] = Travel_encoder.transform(X_train['Type of Travel'])
X_valid['Type of Travel'] = Travel_encoder.transform(X_valid['Type of Travel'])
X_test['Type of Travel'] = Travel_encoder.transform(X_test['Type of Travel'])

Class_encoder = TargetEncoder()
Class_encoder.fit(X_train['Class'], Y_train)
X_train['Class'] = Class_encoder.transform(X_train['Class'])
X_valid['Class'] = Class_encoder.transform(X_valid['Class'])
X_test['Class'] = Class_encoder.transform(X_test['Class'])

In [17]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_train_scaledFrame = pd.DataFrame(X_train_scaled)
X_valid_scaled = scaler.transform(X_valid)
X_valid_scaledFrame = pd.DataFrame(X_valid_scaled)
X_test_scaled = scaler.transform(X_test)
X_test_scaledFrame = pd.DataFrame(X_test_scaled)

In [18]:

Y_train_scaled = np.array(Y_train)

**Creating a model**

In [19]:
model = LogisticRegression(penalty='none')
model.fit(X_train_scaledFrame, Y_train_scaled)

c:\Users\logis\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
c:\Users\logis\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(penalty='none')

In [20]:
print(model.score(X_train_scaledFrame, Y_train))

0.8703478106136304


In [21]:
probs = model.predict_proba(X_train_scaledFrame)

In [22]:
print(probs[:, 1])

[0.14664876 0.11975075 0.05528918 ... 0.3600669  0.44263251 0.27894379]


In [23]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [24]:
print(recall_score(Y_train, model.predict(X_train_scaledFrame)))
print(accuracy_score(Y_train, model.predict(X_train_scaledFrame)))
print(precision_score(Y_train, model.predict(X_train_scaledFrame)))
print(f1_score(Y_train, model.predict(X_train_scaledFrame)))

0.8343858987651993
0.8703478106136304
0.863413206749683
0.8486514125015978


In [25]:
def custom_predict(X, threshold):
    probs = model.predict_proba(X) 
    return (probs[:, 1] > threshold).astype(int)

i = 0.00
mxScore = 0.00
mxThreshold = 0.00
while i < 1.00:
    i = round(i, 3)
    new_preds = custom_predict(X=X_valid_scaledFrame, threshold=i)
    score = f1_score(Y_valid, new_preds)
    if score > mxScore:
        mxScore = score
        mxThreshold = i
    i += 0.001
print(mxScore, mxThreshold)

0.8475829383886255 0.483


0.84743961352657 0.503


**Evaluating with testing data**

In [26]:
import random
from math import floor
from copy import deepcopy

In [27]:
Y_predicted = model.predict(X_test_scaledFrame)
Y_test_array = np.array(Y_test)
Y_test_array.T
li = []
for i in Y_test_array:
    li.append(i[0])
Y_test_array = np.array(li)
print("REAL:", Y_test_array)
print("PREDICTED:", Y_predicted)

REAL: [ True  True  True ... False  True  True]
PREDICTED: [ True  True  True ... False  True  True]


**Bootstrap**


In [28]:
class ChoicesNRandom():    
    def __init__(self, seed=None, n=1):
        import random
        
        self._random_gen = random.Random(seed)
        self._choices_cnt = 0
        self._n = n
        
    def choices(self, *args, **kwargs):
            
        self._choices_cnt += 1
        return self._random_gen.choices(*args, **kwargs)


def round_to_3(x):
    return round(x, 3)


def cut_on_edges(l, fraction_to_cut):  
    to_cut = floor(len(l) * fraction_to_cut)
    return l[to_cut:len(l) - to_cut]


def bootstrap_confidence_interval_solution(y, y_pred, fraction_to_cut, random_gen):
    rnd = random.Random()
    zipped = list(zip(y, y_pred))
    acc, rec, pre, f1 = [], [], [], []
    for n in range(10000):
        y1, y2 = [], []
        obj = random_gen.choices(zipped, k=len(zipped))
        for i in obj:
            y1.append(i[0])
            y2.append(i[1])
        acc.append(accuracy_score(y1, y2))
        rec.append(recall_score(y1, y2))
        pre.append(precision_score(y1, y2))
        f1.append(accuracy_score(y1, y2))

    acc.sort()
    rec.sort()
    pre.sort()
    f1.sort()
    res = []

    cutted = cut_on_edges(acc, fraction_to_cut)
    a, b = cutted[0], cutted[-1]
    res.append((a,b))
    cutted = cut_on_edges(rec, fraction_to_cut)
    a, b = cutted[0], cutted[-1]
    res.append((a,b))
    cutted = cut_on_edges(pre, fraction_to_cut)
    a, b = cutted[0], cutted[-1]
    res.append((a,b))
    cutted = cut_on_edges(f1, fraction_to_cut)
    a, b = cutted[0], cutted[-1]
    res.append((a,b))
    return res

In [29]:
random_gen = ChoicesNRandom(0, n=1000)
print(bootstrap_confidence_interval_solution(Y_test_array, Y_predicted, 0.025, random_gen))   

KeyboardInterrupt: 

[(0.8668925161687712, 0.8742839544194642), (0.8254683758275814, 0.8377523032562065), (0.8602206263418968, 0.8716569767441861), (0.8668925161687712, 0.8742839544194642)]

**RESULTS**

In [ ]:
print("Accuracy:", accuracy_score(Y_test_array, Y_predicted))
print("Recall:", recall_score(Y_test_array, Y_predicted))
print("Precision:", precision_score(Y_test_array, Y_predicted))
print("F1:", f1_score(Y_test_array, Y_predicted))

Accuracy: 0.8706190329534955
Recall: 0.8317188715265502
Precision: 0.8659452296819788
F1: 0.848487034298698
